In [ ]:
import geopandas as gpd
import geopandas
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import platform
from matplotlib import font_manager, rc
import pandas as pd
import folium
import branca
from shapely.geometry.point import Point

In [ ]:
postbox_locate = pd.read_csv('./data/conv_gu_postbox.csv', encoding='cp949')
postbox_locate.head(2)

In [ ]:
r = 500
postbox_locate['geom'] = postbox_locate.apply(lambda r: Point(r['경도'], r['위도']), axis=1)
gdf = gpd.GeoDataFrame(postbox_locate, geometry='geom', crs='epsg:4326')
gdf_flat = gdf.to_crs('epsg:6347')
gdf_flat['geom'] = gdf_flat.geometry.buffer(r)
gdf = gdf_flat.to_crs('epsg:4326')
gdf.head(2)

In [ ]:
u = gdf.iloc[0,3]
for _, row in gdf.iterrows():
    u = u.union(row['geom'])

lats = np.array(gdf['위도'])
lons = np.array(gdf['경도'])

In [ ]:
lat_c = '37.498095'
lon_c = '127.027610'

m = folium.Map([lat_c, lon_c], zoom_start = 13)
for lat, lon in zip(lats, lons):
    folium.Circle(location = [lat, lon], radius = r, color='black', fill_color = 'purple', weight=1).add_to(m)
m.save('./map/postbox_area_covered.html')
m

In [ ]:
nlsp = geopandas.read_file('./data/nlsp_021001003.shp',encoding = 'utf8')
nlsp.head(2)

In [ ]:
nlsp = nlsp.drop(columns=['val'])
nlsp = nlsp.to_crs(epsg=4326)
nlsp = nlsp.fillna(0)
nlsp.head(2)

In [ ]:
def cal_area(poly, file):
    area = poly.area
    intersection = poly.intersection(file).area
    return intersection / area

In [ ]:
m2 = folium.Map([lat_c, lon_c], zoom_start = 13)
for _, row in nlsp.iterrows():
    value = 1 - cal_area(row['geometry'], u)
    sim_geo = gpd.GeoSeries(row['geometry'])
    geo_j = sim_geo.to_json()
    color = plt.cm.OrRd(value)
    color = mpl.colors.to_hex(color)
    
    geo_j = folium.GeoJson(data=geo_j,
                        style_function=lambda feature, color=color: {
                                                                        'fillColor': color,
                                                                        'color':"black",
                                                                        'weight': 2,
                                                                        'dashArray': '5, 5',
                                                                        'fillOpacity': 0.8,
                                                                    })


    folium.Popup(f'{value:.3f}').add_to(geo_j)
    geo_j.add_to(m2)

colormap = branca.colormap.linear.OrRd_06.scale(0, 1)
colormap = colormap.to_step(index=np.linspace(0, 1, 1000))
colormap.caption = 'Area Ratio NOT Covered by postbox_area'
colormap.add_to(m2)
m2.save('./map/postbox_area.html')
m2

**$w_i = lbl \times AreaRatio \times restaurant \times bar \times academy $**

- $w_i$ : 수요량
- $lbl$ : 주거 인구 비례
- $restaurant$ : 식당가
- $bar$ : 유흥주점
- $academy$ : 학원
- $AreaRatio$ : 커버되지않은 면적 비율 (0~1)

In [ ]:
r_met=250
metro = pd.read_csv('./data/conv_gu_metro.csv',
                    encoding='utf-8',
                    sep=",")
metro['geom'] = metro.apply(lambda r_met: Point(r_met['경도'], r_met['위도']), axis=1)
met_gdf = gpd.GeoDataFrame(metro, geometry='geom', crs='epsg:4326')
met_gdf_flat = met_gdf.to_crs('epsg:4326')
met_gdf_flat['geom'] = met_gdf_flat.geometry.buffer(r_met)
met_gdf = met_gdf_flat.to_crs('epsg:4326')

met_u = met_gdf.iloc[0,2]
for _, row in met_gdf.iterrows():
    met_u = met_u.union(row['geom'])

met_lats = np.array(met_gdf['위도'])
met_lons = np.array(met_gdf['경도'])
#######################################################################################
r_pol=500
police = pd.read_csv('./data/conv_police_station.csv',
                    encoding='cp949',
                    sep=",")
police['geom'] = police.apply(lambda r_pol: Point(r_pol['경도'], r_pol['위도']), axis=1)
pol_gdf = gpd.GeoDataFrame(police, geometry='geom', crs='epsg:4326')
pol_gdf_flat = pol_gdf.to_crs('epsg:4326')
pol_gdf_flat['geom'] = pol_gdf_flat.geometry.buffer(r_pol)
pol_gdf = pol_gdf_flat.to_crs('epsg:4326')

pol_u = pol_gdf.iloc[0,3]
for _, row in pol_gdf.iterrows():
    pol_u = pol_u.union(row['geom'])

pol_lats = np.array(pol_gdf['위도'])
pol_lons = np.array(pol_gdf['경도'])
#######################################################################################
r_spol=250
spolice = pd.read_csv('./data/conv_small_police.csv',
                    encoding='cp949',
                    sep=",")
spolice['geom'] = spolice.apply(lambda r_spol: Point(r_spol['경도'], r_spol['위도']), axis=1)
spol_gdf = gpd.GeoDataFrame(spolice, geometry='geom', crs='epsg:4326')
spol_gdf_flat = spol_gdf.to_crs('epsg:4326')
spol_gdf_flat['geom'] = spol_gdf_flat.geometry.buffer(r_spol)
spol_gdf = spol_gdf_flat.to_crs('epsg:4326')

spol_u = spol_gdf.iloc[0,2]
for _, row in spol_gdf.iterrows():
    spol_u = spol_u.union(row['geom'])

spol_lats = np.array(spol_gdf['위도'])
spol_lons = np.array(spol_gdf['경도'])

#######################################################################################
r_dep=250
department = pd.read_csv('./data/conv_gu_dep.csv',
                    encoding='utf-8',
                    sep=",")
department['geom'] = department.apply(lambda r_dep: Point(r_dep['경도'], r_dep['위도']), axis=1)
dep_gdf = gpd.GeoDataFrame(department, geometry='geom', crs='epsg:4326')
dep_gdf_flat = dep_gdf.to_crs('epsg:4326')
dep_gdf_flat['geom'] = dep_gdf_flat.geometry.buffer(r_dep)
dep_gdf = dep_gdf_flat.to_crs('epsg:4326')

dep_u = dep_gdf.iloc[0,2]
for _, row in dep_gdf.iterrows():
    dep_u = dep_u.union(row['geom'])

dep_lats = np.array(dep_gdf['위도'])
dep_lons = np.array(dep_gdf['경도'])

In [ ]:
m3_nlsp = nlsp.replace(to_replace = 'N/A', value = 0)
m3_nlsp = m3_nlsp.astype({'lbl':float})

In [ ]:
m3 = folium.Map([lat_c, lon_c], zoom_start = 13)
values = []
cnt = 0
for _, row in m3_nlsp.iterrows():
    value = (1-cal_area(row['geometry'], u)) * (row['lbl']) * ( 1 - 0.5 * cal_area(row['geometry'], pol_u)) * (1 - 0.3 * cal_area(row['geometry'], spol_u)) * (1 + 0.8 * cal_area(row['geometry'], met_u)) * (1 + 0.6 * cal_area(row['geometry'], dep_u))
    if value >= 0.0005:
        cnt += 1
    else:
        value = 0
    values.append(value)
max_value = max(values)

for i, row in m3_nlsp.iterrows():
    value = values[i]
    sim_geo = gpd.GeoSeries(row['geometry'])
    geo_j = sim_geo.to_json()

    color = plt.cm.Reds(value/max_value)
    color = mpl.colors.to_hex(color)

    if value == 0:
        geo_j = folium.GeoJson(data=geo_j,
                        style_function=lambda feature, color=color: {
                                                                        'fillColor': color,
                                                                        'color':"black",
                                                                        'weight': 0.5,
                                                                        'dashArray': '5, 5',
                                                                        'fillOpacity': 0,
                                                                    })
    else:
        geo_j = folium.GeoJson(data=geo_j,
                        style_function=lambda feature, color=color: {
                                                                        'fillColor': color,
                                                                        'color':"black",
                                                                        'weight': 0.5,
                                                                        'dashArray': '5, 5',
                                                                        'fillOpacity': 0.8,
                                                                    })


    folium.Popup(f'{value:.3f}').add_to(geo_j)
    geo_j.add_to(m3)
    
colormap = branca.colormap.linear.Reds_05.scale(0, max_value)
colormap = colormap.to_step(index=np.linspace(0, max_value, 1000))
colormap.caption = 'Quantity demanded Area'
colormap.add_to(m3)
m3.save('./map/quantity_demanded_area.html')
m3

In [ ]:
print(f'max_value : {max_value:.4f}')
print(f'N_Grid(value>0) : {cnt}, N_Grid(All) : {len(m3_nlsp)}')
print(f'Grid_Ratio : {cnt/len(m3_nlsp):.4f}')

# 2-2. MCLP
- **M**aximal **C**overing **L**ocation **P**roblem

- $\mbox{Maximize} \sum _{i\in I}{w_iy_i}$
-

- Subject to
    - $\sum _{j\in J}{x_j}=K$ : 배치되는 제설함의 최소개수 K
    - $\sum _{j\in N_i}{x_j} \ge y_i \, \forall i \in I$
- 변수
    - $i$ : 수요 지점 index, $i \in I$
    - $j$ : 후보지 index, $j \in J$
    - $w_i$ : 수요량 (value)
    - $S$ : 최대 서비스 거리 (208m)
    - $K$ : 설치할 제설함의 수
    - $d_{ij}$ : 수요 지점 $i$로부터 후보지 $j$까지의 거리
    - $N_i=\{j|d_{ij} \le S \}$ : 수요 지점 $i$로부터 서비스 거리 $S$ 안에 있는 후보지의 집합
    - $x_i = \begin{cases} 1, & \text{제설함이 후보지 }j\text{에 입점하면} \\
    0, & \mbox{otherwise} \end{cases}$
    - $y_j = \begin{cases} 1, & \text{수요 지점 }i\text{가 적어도 하나의 제설함에 의해 서비스되면} \\
    0, & \mbox{otherwise} \end{cases}$

In [ ]:
m3_nlsp['weight'] = values

In [ ]:
m3_nlsp['geo'] = nlsp['geometry'].to_crs(epsg=5179)
m3_nlsp.head()

In [ ]:
def generate_candidate_sites(df):
    sites = []
    for _, row in df.iterrows():
        sites.append([row['geo'].centroid.coords[0][0], row['geo'].centroid.coords[0][1]])
    return np.array(sites)


def generate_candidate_sites(df, M):
    sites = []
    df_sorted = df.sort_values(by='weight', ascending=False)
    for _, row in df_sorted[:M].iterrows():
        sites.append([row['geo'].centroid.coords[0][0], row['geo'].centroid.coords[0][1]])
    return np.array(sites)

In [ ]:
from mip import Model, xsum, maximize, BINARY
from scipy.spatial import distance_matrix
import time
from gurobipy import *

def mclp(df, K, M, radius):
    start = time.time()
    sites = generate_candidate_sites(df, M)
    I = J = sites.shape[0]
    D = distance_matrix(sites, sites)
    mask1 = D <= radius
    D[mask1] = 1
    D[~mask1] = 0

    m = Model()

    x = {}
    y = {}
    
    for i in range(I):
        y[i] = m.addVar(vtype=GRB.BINARY, name=f"y{i}")
    for j in range(J):
        x[j] = m.addVar(vtype=GRB.BINARY, name=f"x{j}")

    m.update()
    
    # 제약식 추가
    m.addConstr(quicksum(x[j] for j in range(J)) == K)

    for i in range(I):
        m.addConstr(quicksum(x[j] for j in np.where(D[i]==1)[0]) >= y[i])

    m.setObjective(quicksum(y[i]for i in range(I)),GRB.MAXIMIZE)
    m.setParam('OutputFlag', 0)
    m.optimize()
    end = time.time()
    
    solution = []
    if m.status == GRB.Status.OPTIMAL:
        for v in m.getVars():
            if v.x==1 and v.varName[0]=="x":
                solution.append(int(v.varName[1:]))
    opt_sites = sites[solution]
    return opt_sites, m.objVal

In [ ]:
radius = 500
K = 10
M = cnt
opt_sites_org, f = mclp(m3_nlsp, K, M, radius)
print('커버되는 수요 지점의 개수 : {}'.format(f))

In [ ]:
opt_df= pd.DataFrame(opt_sites_org, columns=['lon', 'lat'])
opt_df['geom'] = opt_df.apply(lambda r: Point(r['lon'], r['lat']), axis=1)
gdf = gpd.GeoDataFrame(opt_df, geometry='geom', crs='epsg:5179')
gdf = gdf.to_crs(epsg=4329)
gdf['lon'] = gdf['geom'].apply(lambda p:p.x)
gdf['lat'] = gdf['geom'].apply(lambda p:p.y)
gdf.head(2)

In [ ]:
lats = gdf['lat']
lons = gdf['lon']

In [ ]:
m4 = m3
for lat, lon in zip(lats, lons):
    folium.Circle(location = [lat, lon], radius = radius, color='black', fill_color = 'blue', weight=1).add_to(m4)
m4.save('./map/2nd.html')
m4

In [ ]:
radius = 500
M = cnt
K = 0
l = len(m3_nlsp)

new_df = pd.DataFrame(data={'K':[0], 'max_value':[np.round(max_value, 4)], 'mean_value':[np.mean(m3_nlsp['weight'])],
                            'N_Grid(value>0)':cnt, 'Grid_Ratio':np.round(cnt/l,4)})
new_df['K'] = new_df['K'].astype(int)
while True:
    K += 1
    opt_sites_org, f = mclp(m3_nlsp, K, M, radius)
    
    opt_df= pd.DataFrame(opt_sites_org, columns=['lon', 'lat'])
    opt_df['geom'] = opt_df.apply(lambda row: Point(row['lon'], row['lat']), axis=1)
    gdf = gpd.GeoDataFrame(opt_df, geometry='geom', crs='epsg:5179')
    gdf = gdf.to_crs('epsg:4326')
    gdf['lon'] = gdf['geom'].apply(lambda p:p.x)
    gdf['lat'] = gdf['geom'].apply(lambda p:p.y)
    lats = gdf['lat']
    lons = gdf['lon']
    opt_df = gpd.GeoDataFrame(opt_df, geometry='geom', crs='epsg:5179')
    opt_df['geometry'] = opt_df.geom.buffer(r)
    gdf['circle'] = opt_df['geometry'].to_crs('epsg:4326')

    u_tmp = u
    for c in gdf['circle']:
        u_tmp = u_tmp.union(c)

    values = []
    for _, row in m3_nlsp.iterrows():
        value = value = (1-cal_area(row['geometry'], u)) * (row['lbl']) * ( 1 - 0.5 * cal_area(row['geometry'], pol_u)) * (1 - 0.3 * cal_area(row['geometry'], spol_u)) * (1 + 0.8 * cal_area(row['geometry'], met_u)) * (1 + 0.6 * cal_area(row['geometry'], dep_u))
        if value < 0.0005:
            value = 0
        values.append(value)
    max_value = max(values)
    
    tmp_df = pd.DataFrame({'K':[int(K)], 'max_value':[np.round(max_value, 4)], 'mean_value':[np.mean(values)],
                        'N_Grid(value>0)':[cnt-f], 'Grid_Ratio':[np.round((cnt-f)/l,4)]},
                         index=[K])
    new_df = pd.concat([new_df, tmp_df])

    if f == M:
        break

print(f'최소 {K}개의 여성 안심 택배함으로 모든 취약 지점을 커버할 수 있다.')
new_df.to_csv('./result.csv')

In [ ]:
m5 = folium.Map([lat_c, lon_c], zoom_start = 13)
values = m3_nlsp['weight']
max_value = max(values)
for i, row in m3_nlsp.iterrows():
    value = values[i]
    sim_geo = gpd.GeoSeries(row['geometry'])
    geo_j = sim_geo.to_json()

    color = plt.cm.Reds(value/max_value)
    color = mpl.colors.to_hex(color)

    if value == 0:
        geo_j = folium.GeoJson(data=geo_j,
                        style_function=lambda feature, color=color: {
                                                                        'fillColor': color,
                                                                        'color':"black",
                                                                        'weight': 0.5,
                                                                        'dashArray': '5, 5',
                                                                        'fillOpacity': 0,
                                                                    })
    else:
        geo_j = folium.GeoJson(data=geo_j,
                        style_function=lambda feature, color=color: {
                                                                        'fillColor': color,
                                                                        'color':"black",
                                                                        'weight': 0.5,
                                                                        'dashArray': '5, 5',
                                                                        'fillOpacity': 0.8,
                                                                    })


    folium.Popup(f'{value:.3f}').add_to(geo_j)
    geo_j.add_to(m5)
colormap = branca.colormap.linear.Reds_05.scale(0, max_value)
colormap = colormap.to_step(index=np.linspace(0, max_value, 1000))
colormap.caption = 'lbl × Area Ratio not covered by jae-seor-ham'
colormap.add_to(m5)
for lat, lon in zip(lats, lons):
    folium.Circle(location = [lat, lon], radius = radius, color='black', fill_color = 'blue', weight=1).add_to(m5)
m5.save('./map/final.html')
m5